In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import PydanticOutputParser
from typing import List
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document
from langchain.output_parsers import OutputFixingParser
from operator import itemgetter
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI  # example LLM
from langchain.output_parsers import StructuredOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnablePassthrough, chain, RunnableConfig
import pandas as pd
import re
import os
from langchain_openai import OpenAI
from dotenv import load_dotenv
load_dotenv()




True

In [4]:

llm = OpenAI(name="gpt-4o")


In [5]:
llm.invoke("Hello, world!")

"'\n\n\n\n\nHello, world!"

In [6]:
llm = OpenAI(name="o3")

In [7]:
llm.invoke("Hello, world!")

"\n\nI'm an AI created by OpenAI. I'm constantly learning and evolving, so my responses may change over time. How can I assist you today?"

In [8]:
prompt = PromptTemplate.from_template("""
    How to say {input} in {target_language} language
""")           

chain = prompt | llm | StrOutputParser()


chain.invoke({"input": "Danke", "target_language": "Russian"})






'\nСпасибо (Spasibo)'

In [9]:
import o
print(os.getcwd())  # покажет текущую папку
print(os.listdir()) # покажет, что в ней лежит


ModuleNotFoundError: No module named 'o'

In [10]:
def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents

folder_path = "documents"
documents = load_documents(folder_path)

len(documents)

 

42

In [11]:
documents

[Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 14.0 (Windows)', 'creationdate': '2022-10-25T14:23:09+03:00', 'moddate': '2022-10-25T14:23:12+03:00', 'trapped': '/False', 'source': 'documents/_upload_iblock_723_v18v3w60o611z0grp7svpcq0ul5nv76p.pdf', 'total_pages': 5, 'page': 0, 'page_label': '1'}, page_content='ИНСТРУКЦИЯ\nТиволи 4 см «римская кладка»\nпо укладке тротуарной плитки'),
 Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 14.0 (Windows)', 'creationdate': '2022-10-25T14:23:09+03:00', 'moddate': '2022-10-25T14:23:12+03:00', 'trapped': '/False', 'source': 'documents/_upload_iblock_723_v18v3w60o611z0grp7svpcq0ul5nv76p.pdf', 'total_pages': 5, 'page': 1, 'page_label': '2'}, page_content='1. Для создания рисунка “римской кладки” используйте одно из шести сочетаний \nэлементов тротуарных плит Тиволи производства компании White Hills.\nВариант 1\nВариант 2'),
 Document(metadata={'producer': 'Adobe PDF Library 15.

In [12]:
[doc.metadata for doc in documents]

[{'producer': 'Adobe PDF Library 15.0',
  'creator': 'Adobe InDesign 14.0 (Windows)',
  'creationdate': '2022-10-25T14:23:09+03:00',
  'moddate': '2022-10-25T14:23:12+03:00',
  'trapped': '/False',
  'source': 'documents/_upload_iblock_723_v18v3w60o611z0grp7svpcq0ul5nv76p.pdf',
  'total_pages': 5,
  'page': 0,
  'page_label': '1'},
 {'producer': 'Adobe PDF Library 15.0',
  'creator': 'Adobe InDesign 14.0 (Windows)',
  'creationdate': '2022-10-25T14:23:09+03:00',
  'moddate': '2022-10-25T14:23:12+03:00',
  'trapped': '/False',
  'source': 'documents/_upload_iblock_723_v18v3w60o611z0grp7svpcq0ul5nv76p.pdf',
  'total_pages': 5,
  'page': 1,
  'page_label': '2'},
 {'producer': 'Adobe PDF Library 15.0',
  'creator': 'Adobe InDesign 14.0 (Windows)',
  'creationdate': '2022-10-25T14:23:09+03:00',
  'moddate': '2022-10-25T14:23:12+03:00',
  'trapped': '/False',
  'source': 'documents/_upload_iblock_723_v18v3w60o611z0grp7svpcq0ul5nv76p.pdf',
  'total_pages': 5,
  'page': 2,
  'page_label': '3'}

In [13]:
os.listdir("documents")

['_upload_iblock_723_v18v3w60o611z0grp7svpcq0ul5nv76p.pdf',
 '_upload_iblock_606_nlf798sqxiddt0jddmfg8uy6r1qec2tp.pdf',
 '_upload_iblock_023_0235e4bc383926f54e57888839843405.pdf',
 '_upload_iblock_e5b_e5bc5b097ef730e4719c4a20e4591d6f.pdf',
 '_upload_iblock_1f9_1f9e55bb3fc71c70f3e02d3d144682c0.pdf',
 '_upload_iblock_d18_d18ed70db7b40893f7bd2a69922658e6.pdf',
 '_upload_iblock_ce6_ce62808ad9b1e4736343d7668b99fd5b.pdf',
 '_upload_iblock_d7e_d7e210e9d0593b74f8f96866b28a4ab9.pdf',
 '_upload_iblock_f59_f1aphtbdgbjobtc383px49qnexz8erep.pdf',
 '_upload_iblock_6c0_56uylwl41jxefdrlk9i8d1lmt6pbwrr3.pdf',
 '_upload_iblock_955_955d9f1eee6a824fdd9312beed408a12.pdf',
 '_upload_iblock_c1b_c1b39be5c349c4ffd1459260c0ae0b8c.pdf']

In [14]:
# text classification
from langchain_text_splitters import RecursiveCharacterTextSplitter

documents = load_documents("documents")
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(documents)
len(chunks)

56

In [15]:

auto_label_prompt = PromptTemplate.from_template("""
Come up with a short, general topic name for the following text.

Text:
{chunk}

Return only the topic name.
""")
auto_label_chain = auto_label_prompt | llm | StrOutputParser()

results = []

for chunk in chunks[:10]:  
    topic = auto_label_chain.invoke({
        "chunk": chunk.page_content
    })
    results.append({
        "text": chunk.page_content[:50],
        "topic": topic
    })

import pandas as pd
df = pd.DataFrame(results)
df.tail()


,text,topic
5,продукции. Тротуары должны быть выполнены в со...,"\n""Тротуары и продукция"""
6,ГИДРОФОБИЗАТОР\nВодоотталкивающий кремнийорган...,\nWater-repellent silicon-organic composition ...
7,паропроницаемости. Глубина проникновения раств...,"\n""Waterproof Coating Properties and Usage Ins..."
8,Способ применения\nПодготовка поверхности\nПов...,\nSurface Preparation and Application
9,"элемент, проверить соответствие цвета обработа...",\nSurface Coating and Application Safety


In [ ]:
class Book(self, ):
